In [4]:
import pandas as pd


dataset = pd.read_csv("all1819.csv")

ban = dataset[dataset['class'] =="['বাংলাদেশ']"]
sports = dataset[dataset['class'] =="['খেলা']"]
interN = dataset[dataset['class'] =="['আন্তর্জাতিক']"]
dur = dataset[dataset['class'] =="['দূর পরবাস']"]
enet = dataset[dataset['class'] =="['বিনোদন']"]
other = dataset[dataset['class'] =="['']"]


dataset.drop(dataset.index[dataset['class'] =="['অর্থনীতি']"],inplace = True)
dataset.drop(dataset.index[dataset['class'] =="['মতামত']"],inplace = True)
dataset.drop(dataset.index[dataset['class'] =="['জীবনযাপন']"],inplace = True)
dataset.drop(dataset.index[dataset['class'] =="['বিজ্ঞান ও প্রযুক্তি']"],inplace = True)
dataset.drop(dataset.index[dataset['class'] =="['শিক্ষা']"],inplace = True)
dataset.drop(dataset.index[dataset['class'] =="['আমরা']"],inplace = True)
dataset.drop(dataset.index[dataset['class'] =="['পাঁচমিশালি']"],inplace = True)
dataset.drop(dataset.index[dataset['class'] =="['বিশেষ সংখ্যা']"],inplace = True)
dataset.drop(dataset.index[dataset['class'] =="['তারুণ্যের জয়োৎসব']"],inplace = True)
dataset.drop(dataset.index[dataset['class'] =="['ফিচাররস+আলো']"],inplace = True)

print(dataset['class'].value_counts())




x = ban[:]
x = x.append(sports[:])
x= x.append(interN[:])
x= x.append(dur[:])
x= x.append(enet[:])
x= x.append(other[:])



docs = x["Document"][:]


cl = [xs.replace('ইহাতে মন্তব্য প্রদান বন্ধ রয়েছে                                |    ', '') for xs in docs]
cl2 = [xs.replace('\xa0', '').replace('\u200c', '') for xs in cl]
cl3 = [xs.replace('?', u'।').replace('!', u'।').replace(';', u'।') for xs in cl]

sentTok =[]
wordTok = []
for doc in cl3:
    sentTok.append(doc.split('।'))



import re
pnList = ['.', ',', ':', '(', ')', '[', ']', '{', '}', '`', '*', '&', '‘', '’', '–']
cl4 = [xs.replace(u'।'," ") for xs in cl3]

cl5=[[] for x in range(len(cl4))]
for i in range(len(cl4)):
#     cl5[i] = cl4[i].replace("(","").replace(")","").replace(",","").replace("+","").replace(")","")
    cl5[i] = re.sub("[০১২৩৪৫৬৭৮৯]",'',cl4[i])
    cl5[i] = re.sub("[,./<>)(}{_*&^%$#@~`]",'',cl5[i])

for doc in cl5:
    wordTok.append(doc.split())


#remove stopword
stRem = [[] for i in range(len(wordTok))]

def remStWords(punctuF):
    stWord = open("./stopwords-bn.txt", encoding='utf8')
    stWordFile = [i.rstrip() for i in stWord]
    filtStWord = [w for w in punctuF if w not in stWordFile]

    return filtStWord
for i in range(len(wordTok)):
    stRem[i] = remStWords(wordTok[i])
    

print(len(stRem[1]))



print(len(stRem))
allWord = []
for i in range(len(stRem)):
    allWord+=stRem[i]
 


print(len(allWord))
# ##Term frequency    
# from collections import Counter
# counts = Counter(allWord)
# # print(counts['অনেক'])
# tfDict = {}
# for sent in stRem:
#     for word in sent:
#         tfDict[word] = counts[word]/len(sent)
# #print(token)
# #print(counts)
# #print(tfDict)
        
# #number of documents
# n = len(stRem)
# print(n)
# stRem
# #Number of terms
# terms = list(tfDict.keys())
# #print(len(terms))
# #
# #Calculate Inverse Documentt frequency
# import math
# idfDict = {}
# nnn = 0
# for term in terms:
#     nnn+=1
#     print(nnn)
#     count = 0
#     for doc in stRem:
#         if term in doc:
#             count+=1
#     idfDict[term] = math.log(n/(count+1))
  
#  #Calculate tfIdf
# tfIdf = {}

# for term in terms:
#     tfIdf[term] = round(tfDict[term] * idfDict[term],4)

# tfIdfSorted = {k:v for k,v in sorted(tfIdf.items(), key = lambda item: item[1], reverse = True)}

# print(tfIdfSorted)

['বাংলাদেশ']       6348
['খেলা']           2247
['আন্তর্জাতিক']    1456
['দূর পরবাস']       853
['বিনোদন']          789
['']                487
Name: class, dtype: int64
512
12180
3008504


In [7]:
import pickle as pk
inp = open('F:\dataP\\tfidfSorted','rb')

tfIdfSorted = pk.load(inp)
inp.close()


In [7]:

import random
import numpy as np
from sklearn.metrics import f1_score
#Import model for NB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split 
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn import preprocessing
import itertools
tfIdfSortedS = dict(itertools.islice(tfIdfSorted.items(),5000))

# create classifier object gor model
model=GaussianNB()

key = list(tfIdfSortedS.keys())
len(key)


5000

In [16]:


def create_starting_population(individuals, chromosome_length):
    geneNo = chromosome_length
    populationSize = individuals
    # Defining the population size.

    popSize = (populationSize,geneNo) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.

    #Creating the initial population.
    newPop = np.random.random_integers(low=0, high=1, size=popSize)

    
    return newPop

def calculate_fitness(population):
    keyS = [[] for x in range(10)]
    for j in range(len(population)):
        for i in range(chromosome_length):
            if population[j][i]==1:
                keyS[j].append(key[i])

            
    tfIdff = [[[] for i in range(len(stRem))] for j in range(len(keyS))]
    for i in range(len(stRem)):
        for j in range(len(keyS)):
            for word in keyS[j]:
                    if word in stRem[i]:
                        tfIdff[j][i].append(tfIdfSorted[word])
                    else:
                        tfIdff[j][i].append(0)
    fitnessS =[]
    for j in range(len(keyS)-1):
        dataX = tfIdff[j][:]
        dataY = x['class'][:1200]
        try:
            trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size = 0.2, random_state = 0,stratify=dataY)
            #print(len(trainX),len(trainY),(trainX[0]))
            model.fit(trainX, trainY)

            # make predictions on training data
            predictions_train = model.predict(trainX)
            print(j,'\nTraining Accuracy using DT :',accuracy_score(trainY, predictions_train))

            # make predictions on test data
            predictions_test = model.predict(testX)
            print(j,'\nTest Accuracy using DT:',accuracy_score(testY, predictions_test))
            fitnessS.append(f1_score(testY,predictions_test,average='macro'))
        except:
            fitnessS.append(0)
            
    return fitnessS

def select_individual_by_tournament(population, scores):
    # Get population size
    population_size = len(scores)
    
    # Pick individuals for tournament
    fighter_1 = random.randint(0, population_size-1)
    fighter_2 = random.randint(0, population_size-1)
    
    # Get fitness score for each
    fighter_1_fitness = scores[fighter_1]
    fighter_2_fitness = scores[fighter_2]
    
    # Identify undividual with highest fitness
    # Fighter 1 will win if score are equal
    if fighter_1_fitness >= fighter_2_fitness:
        winner = fighter_1
    else:
        winner = fighter_2
    
    # Return the chromsome of the winner
    return population[winner, :]

def breed_by_crossover(parent_1, parent_2):
    # Get length of chromosome
    chromosome_length = len(parent_1)
    
    # Pick crossover point, avoding ends of chromsome
    crossover_point = random.randint(1,chromosome_length-1)
    
    # Create children. np.hstack joins two arrays
    child_1 = np.hstack((parent_1[0:crossover_point],
                        parent_2[crossover_point:]))
    
    child_2 = np.hstack((parent_2[0:crossover_point],
                        parent_1[crossover_point:]))
    if list(child_1).count(1.0) == 0 or list(child_2).count(1.0) == 0:
        print('piku')
        breed_by_crossover(parent_1, parent_2)
        
    # Return children
    return child_1, child_2

def randomly_mutate_population(population, mutation_probability):
    
    # Apply random mutation
        random_mutation_array = np.random.random(
            size=(population.shape))
        
        random_mutation_boolean = \
            random_mutation_array <= mutation_probability

        population[random_mutation_boolean] = \
        np.logical_not(population[random_mutation_boolean])
        
        # Return mutation population
        return population

In [20]:
# Set general parameters
chromosome_length = len(tfIdfSorted)
population_size = 10
maximum_generation = 100
best_score_progress = [] # Tracks progress

# Create reference solution 
# (this is used just to illustrate GAs)
#reference = create_reference_solution(chromosome_length)

# Create starting population
population = create_starting_population(population_size, chromosome_length)

# Display best score in starting population
scores = calculate_fitness( population)
best_score = np.max(scores) * 100
print ('Starting best score, percent target: %.1f' %best_score)

# Add starting best score to progress tracker
best_score_progress.append(best_score)



# Now we'll go through the generations of genetic algorithm
for generation in range(maximum_generation):
    # Create an empty list for new population
    new_population = []
    
    # Create new popualtion generating two children at a time
    for i in range(int(population_size/2)):
        parent_1 = select_individual_by_tournament(population, scores)
        parent_2 = select_individual_by_tournament(population, scores)
        child_1, child_2 = breed_by_crossover(parent_1, parent_2)
        new_population.append(child_1)
        new_population.append(child_2)
    
    # Replace the old population with the new one
    population = np.array(new_population)
    
    # Apply mutation
    mutation_rate = 0.002
    population = randomly_mutate_population(population, mutation_rate)
    for pop in population:
        if list(pop).count(1.0) == 0:
            print('here')
        
    # Score best solution, and add to tracker
    scores = calculate_fitness(population)
    best_score = np.max(scores) * 100
    best_score_progress.append(best_score)

# GA has completed required generation
print ('End best score, percent target: %.1f' %best_score)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  if __name__ == '__main__':


IndexError: list index out of range

In [ ]:

# Plot progress
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(best_score_progress)
plt.xlabel('Generation')
plt.ylabel('Best score (% target)')
plt.show()
print(list(population[0]).count(1))